In [1]:
'''
This is Homework 5 for AMATH 581
Tianbo Zhang 1938501
'''
import numpy as np
import matplotlib.pyplot as plt
import time
import random
from scipy.fft import fft2, ifft2
from scipy.integrate import solve_ivp
from scipy.sparse import spdiags
from scipy.linalg import lu, solve_triangular
from scipy.sparse.linalg import bicgstab, gmres, spsolve

In [2]:

# Parameters
m = 64  # Grid size per dimension
n = m * m  # Total number of grid points
L = 10  # Domain size
dx = (2 * L) / m  # Grid spacing

# Base vectors for diagonals
ones = np.ones(n)
zeros = np.zeros(n)

# Periodic wrap diagonals: farthest connections
periodic_shift_m1 = np.zeros(n)
periodic_shift_p1 = np.ones(n)
periodic_shift_m1[m-1::m] = 1
periodic_shift_p1[m-1::m] = 0
periodic_shift_m1_re = np.concatenate((periodic_shift_m1[-1:], periodic_shift_m1[:-1]))
periodic_shift_p1_re = np.concatenate((periodic_shift_p1[-1:], periodic_shift_p1[:-1]))


# Construct derivative matrix A1 (Laplacian)
diagonals_a = [-4 * ones, periodic_shift_p1, periodic_shift_p1_re, 
               periodic_shift_m1_re, periodic_shift_m1, 
               ones, ones, ones, ones]
offsets_a = [0, -1, 1, -(m - 1), m - 1, m, -m, -(n - m), (n - m)]

A = spdiags(diagonals_a, offsets_a, n, n).toarray() / dx**2
A[0,0] = -4

# Construct derivative matrix A2 (x-derivative)
diagonals_b = [ones, -ones, -ones, ones]
offsets_b = [m, -m, n - m, -(n - m)]

B = spdiags(diagonals_b, offsets_b, n, n).toarray() / (2 * dx)

# Construct derivative matrix A3 (y-derivative)
diagonals_c = [periodic_shift_m1_re, -periodic_shift_m1, -periodic_shift_p1, periodic_shift_p1_re]
offsets_c = [-(m - 1), m - 1, -1, 1]

C = spdiags(diagonals_c, offsets_c, n, n).toarray() / (2 * dx)

In [3]:
# Set up initial conditions
Lx = 20    # spatial domain of x
Ly = 20    # spatial domain of y
nx = 64   # number of discretization points in x
ny = 64   # number of discretization points in y
N = nx * ny   # elements in reshaped initial condition
nu = 0.001
tspan = np.arange(0, 4.5, 0.5)    # time span

# Set up domains
x2 = np.linspace(-Lx/2, Lx/2, nx+1) # x domain
x = x2[:nx]   
y2 = np.linspace(-Ly/2, Ly/2, ny+1) # y domain
y = y2[:ny]  
X, Y = np.meshgrid(x, y)  # make 2D

# Set up Gaussian Elliptical initial condition
w = 1 * np.exp(-X**2 - (Y**2)/20)
w0 = w.reshape(N)

In [4]:
# Part (a): FFT method
# Define spectral k values
kx = (2 * np.pi / Lx) * np.concatenate((np.arange(0, nx/2), np.arange(-nx/2, 0)))
kx[0] = 1e-6
ky = (2 * np.pi / Ly) * np.concatenate((np.arange(0, ny/2), np.arange(-ny/2, 0)))
ky[0] = 1e-6
KX, KY = np.meshgrid(kx, ky)
K = KX**2 + KY**2

# FFT method
def spc_rhs(t, w0, nx, ny, nu, A, B, C, K, N):
    wt = fft2(w0.reshape(nx, ny))
    phi_t = -wt / K
    phi = np.real(ifft2(phi_t)).reshape(N)
    rhs = nu * np.dot(A, w0) + np.dot(B, w0) * np.dot(C, phi) - np.dot(B, phi) * np.dot(C, w0)
    return rhs

start_time = time.time()
sol_1 = solve_ivp(spc_rhs, [tspan[0], tspan[-1]], w0, t_eval = tspan, args = (nx, ny, nu, A, B, C, K, N), method = 'RK45')
A1 = sol_1.y
end_time = time.time()
elapsed_time_fft = end_time - start_time
print(f"Elapsed time (fft): {elapsed_time_fft:.2f} seconds")

Elapsed time (fft): 0.37 seconds


In [5]:
# Part (b)
A[0,0] = 2

# Direct solve method
def direct_solve(t, w0, nu, A, B, C):
    phi = spsolve(A, w0)
    rhs = nu * np.dot(A, w0) + np.dot(B, w0) * np.dot(C, phi) - np.dot(B, phi) * np.dot(C, w0)
    return rhs
    
start_time = time.time()
sol_2 = solve_ivp(direct_solve, [tspan[0], tspan[-1]], w0, t_eval = tspan, args = (nu, A, B, C), method = 'RK45')
A2 = sol_2.y
end_time = time.time()
elapsed_time_ds = end_time - start_time
print(f"Elapsed time (direct solve): {elapsed_time_ds:.2f} seconds")

C:\Users\2019z\AppData\Local\Temp\ipykernel_25756\1942364065.py:6: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  phi = spsolve(A, w0)


Elapsed time (direct solve): 4.45 seconds


In [6]:
# LU Decomposition
# Get LU decomposition of A
P, L, U = lu(A)
def lu_solve(t, w0, nu, P, L, u):
    Pb = np.dot(P, w0)
    y = solve_triangular(L, Pb, lower=True)
    phi = solve_triangular(U, y)
    rhs = nu * np.dot(A, w0) + np.dot(B, w0) * np.dot(C, phi) - np.dot(B, phi) * np.dot(C, w0)
    return rhs
    
start_time = time.time()
sol_3 = solve_ivp(lu_solve, [tspan[0], tspan[-1]], w0, t_eval = tspan, args = (nu, P, L, U), method = 'RK45')
A3 = sol_3.y
end_time = time.time()
elapsed_time_lu = end_time - start_time
print(f"Elapsed time (LU decomposition): {elapsed_time_lu:.2f} seconds")

Elapsed time (LU decomposition): 1.08 seconds


In [7]:
# BICSTAB
def stab_solve(t, w0, nu, A, B, C):
    phi, info = bicgstab(A, w0, tol=1e-6, maxiter=1000)
    rhs = nu * np.dot(A, w0) + np.dot(B, w0) * np.dot(C, phi) - np.dot(B, phi) * np.dot(C, w0)
    return rhs

start_time = time.time()
sol_4 = solve_ivp(stab_solve, [tspan[0], tspan[-1]], w0, t_eval = tspan, args = (nu, A, B, C), method = 'RK45')
A4 = sol_4.y
end_time = time.time()
elapsed_time_stab = end_time - start_time
print(f"Elapsed time (BICGSTAB): {elapsed_time_stab:.2f} seconds")

# GMRES
def gmres_solve(t, w0, nu, A, B, C):
    phi, info = gmres(A, w0, tol=1e-6, restart=50, maxiter=1000)
    rhs = nu * np.dot(A, w0) + np.dot(B, w0) * np.dot(C, phi) - np.dot(B, phi) * np.dot(C, w0)
    return rhs

start_time = time.time()
sol_5 = solve_ivp(gmres_solve, [tspan[0], tspan[-1]], w0, t_eval = tspan, args = (nu, A, B, C), method = 'RK45')
A5 = sol_5.y
end_time = time.time()
elapsed_time_gmres = end_time - start_time
print(f"Elapsed time (GMRES): {elapsed_time_gmres:.2f} seconds")

C:\Users\2019z\AppData\Local\Temp\ipykernel_25756\2501249209.py:3: DeprecationWarning: 'scipy.sparse.linalg.bicgstab' keyword argument `tol` is deprecated in favor of `rtol` and will be removed in SciPy v1.14.0. Until then, if set, it will override `rtol`.
  phi, info = bicgstab(A, w0, tol=1e-6, maxiter=1000)


Elapsed time (BICGSTAB): 52.28 seconds


C:\Users\2019z\AppData\Local\Temp\ipykernel_25756\2501249209.py:16: DeprecationWarning: 'scipy.sparse.linalg.gmres' keyword argument `tol` is deprecated in favor of `rtol` and will be removed in SciPy v1.14.0. Until then, if set, it will override `rtol`.
  phi, info = gmres(A, w0, tol=1e-6, restart=50, maxiter=1000)


Elapsed time (GMRES): 65.29 seconds


In [8]:
# Part (c)
# Two opposite charged vorticies:
w_opposite = 1 * np.exp(-X**2 - (Y**2)/20) - 1 * np.exp(-(X-1)**2 - ((Y-2)**2)/20)
w_opposite = w_opposite.reshape(N)

# Same charged vorticies:
w_same = 1 * np.exp(-X**2 - (Y**2)/20) + 1 * np.exp(-(X-1)**2 - ((Y-2)**2)/20)
w_same = w_same.reshape(N)

# Two pairs of opposite charged vorticies than can be made to collide
w_pairs = 1 * np.exp(-X**2 - (Y**2)/20) - 1 * np.exp(-(X-1)**2 - ((Y-2)**2)/20) + 1 * np.exp(-X**2 - (Y**2)/20) - 1 * np.exp(-(X-1)**2 - ((Y-2)**2)/20)
w_pairs = w_opposite.reshape(N)

# Random assortment of vorticies
w_rand = 1 * np.exp(-X**2 - (Y**2)/20)
for i in range (10):
    rand_strength = random.random()
    rand_x = random.random()
    rand_y = random.random()
    w_rand += rand_strength * np.exp(-(X + rand_x)**2 - ((Y + rand_y)**2)/20)
w_rand = w_rand.reshape(N)

def spc_rhs(t, w0, nx, ny, nu, A, B, C, K, N):
    wt = fft2(w0.reshape(nx, ny))
    phi_t = -wt / K
    phi = np.real(ifft2(phi_t)).reshape(N)
    rhs = nu * np.dot(A, w0) + np.dot(B, w0) * np.dot(C, phi) - np.dot(B, phi) * np.dot(C, w0)
    return rhs

start_time = time.time()
sol_opposite = solve_ivp(spc_rhs, [tspan[0], tspan[-1]], w_opposite, t_eval = tspan, args = (nx, ny, nu, A, B, C, K, N), method = 'RK45')
end_time = time.time()
elapsed_time_op = end_time - start_time
print(f"Elapsed time (opposite vorticies usiing fft): {elapsed_time_op:.2f} seconds")

start_time = time.time()
sol_same = solve_ivp(spc_rhs, [tspan[0], tspan[-1]], w_same, t_eval = tspan, args = (nx, ny, nu, A, B, C, K, N), method = 'RK45')
end_time = time.time()
elapsed_time_sa = end_time - start_time
print(f"Elapsed time (next to each other vorticies usiing fft): {elapsed_time_sa:.2f} seconds")

start_time = time.time()
sol_pairs = solve_ivp(spc_rhs, [tspan[0], tspan[-1]], w_pairs, t_eval = tspan, args = (nx, ny, nu, A, B, C, K, N), method = 'RK45')
end_time = time.time()
elapsed_time_pa = end_time - start_time
print(f"Elapsed time (paired vorticies usiing fft): {elapsed_time_pa:.2f} seconds")

start_time = time.time()
sol_rand = solve_ivp(spc_rhs, [tspan[0], tspan[-1]], w_rand, t_eval = tspan, args = (nx, ny, nu, A, B, C, K, N), method = 'RK45')
end_time = time.time()
elapsed_time_rand = end_time - start_time
print(f"Elapsed time (random vorticies usiing fft): {elapsed_time_rand:.2f} seconds")

Elapsed time (opposite vorticies usiing fft): 0.35 seconds
Elapsed time (next to each other vorticies usiing fft): 0.63 seconds
Elapsed time (paired vorticies usiing fft): 0.33 seconds
Elapsed time (random vorticies usiing fft): 2.37 seconds


In [9]:
# Part (d)

